In [1]:
%matplotlib inline
import matplotlib as plt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1 import ImageGrid
import cv2
import tensorflow_datasets as tfds
import os

## Layer 1: fucntions to visualize layer 1 with its effect on batch images

In [33]:
def fmlayer1(n_filters, img, n_batch, model):
    
    
    #Number of filters to show
    num_filter_to_show = n_filters
    
    #Setting the grid
    fig = plt.figure(figsize=(6., 108.))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(num_filter_to_show, 3),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                     )

    #Iterate over the filters
    for i in range(num_filter_to_show):
        
        #Load the image of the result (feature map)
        result_f = mpimg.imread('results/{}/feature_maps/fmp_layer_1_num_kernel_{}_batch_{}.jpg'.format(model,i,n_batch)) #/ 255.0
        
        #load the shape of the feature map to resize the other images to it
        resize_w = result_f.shape[1]
        resize_h = result_f.shape[0]
      
        #Original
#         img_original = mpimg.imread(img) / 255.0
        img_original = img
        img_original = cv2.resize(img_original, (resize_h, resize_w))

        #Filter
        tmp = cv2.imread('results/{}/kernels/kn_layer_1_num_kernel_{}.jpg'.format(model,i))
        cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
        filt = cv2.resize(tmp,(resize_h,resize_w))
        
        #iterate over the grid
        for idx, (_, im) in enumerate(zip(grid, [img_original, result_f, filt])):
            # Iterating over the grid returns the Axes.
            ax = grid[3*i+idx]
            ax.axis('off')
            if(idx == 1):
                ax.imshow(im,cmap='gray')
            else:
                ax.imshow(im)
    
    directory = f'results/{model}/visualizations'
    if not os.path.exists(directory):
        os.makedirs(directory)

    plt.savefig("results/{}/visualizations/layer_1_batch_{}.svg".format(model,n_batch), bbox_inches='tight')
#     plt.show()
    plt.close()

In [34]:
# ds = tfds.load('cifar10', split='train', as_supervised=True)
# ds = ds.take(32)
def get_fmlayer1(model, n_batch, n_kernels):
    ori_path = f'results/{model}/originals/original_nbatch_'
    originals = []
    for i in range(n_batch):
        tmp_img = cv2.cvtColor(cv2.imread(ori_path + str(i) + '.jpg'), cv2.COLOR_BGR2RGB)
        fmlayer1(n_kernels, tmp_img, i, model)


## Layer n: Function to visualize Layer n and its effect over batch images

First, we have a function to save or stage an img from a fig

In [35]:
# define a function which returns an image as numpy array from figure
import io
def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi, bbox_inches='tight')
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return img

Now, we have a function that retrieve all the feature maps of the last layer

In [36]:
def fmlayern_0(n_filters, cols, layer, n_batch,model):    #n_filters, img, layer, n_batch, n_ch=1):
    num_filter_to_show = n_filters

    fig = plt.figure(figsize=(40., 54.))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=((num_filter_to_show//cols)+1, cols ),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                     )
    cont = 0
    for i in range((num_filter_to_show//cols)+1):
        #Original
#         img_original = mpimg.imread(img) / 255.0

        
        aux = []
        for j in range(cols):
            
            if (cont < n_filters):
                #Load the image test and normalize
                result_f = mpimg.imread('results/{}/feature_maps/fmp_layer_{}_num_kernel_{}_batch_{}.jpg'.format(model,layer-1,cont,n_batch)) #/ 255.0
                result_f = cv2.resize(result_f, (100,100))
                cont += 1
            
            else:
            
                result_f = np.zeros([100,100,3],dtype=np.uint8)
                result_f.fill(255) # or img[:] = 255
            
            aux.append(result_f)
            
        for idx, (_, im) in enumerate(zip(grid, aux )):
            # Iterating over the grid returns the Axes.
            ax = grid[cols*i+idx]
            ax.axis('off')
            ax.imshow(im,cmap='gray')

#     plt.show()
#     plt.axis('off')
    data = get_img_from_fig(fig)
    plt.close()
    
    return data

The funtion below retrieves all the channels of the kernel that will filter the corresponding feature map.

In [37]:
def fmlayern_1(n_filters, cols, layer, n_kn, model):    #n_filters, img, layer, n_batch, n_ch=1):
    num_filter_to_show = n_filters

    fig = plt.figure(figsize=(40., 54.))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=((num_filter_to_show//cols)+1, cols ),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                     )
    cont = 0
    for i in range((num_filter_to_show//cols)+1):
        #Original
#         img_original = mpimg.imread(img) / 255.0

        
        aux = []
        for j in range(cols):
            
            if (cont < n_filters):
                #Load the image test and normalize
                result_f = mpimg.imread('results/{}/kernels/kn_layer_{}_num_kernel_{}_ch_{}.jpg'.format(model,layer,n_kn,cont))
                result_f = cv2.resize(result_f, (100,100))
                cont += 1
            
            else:
            
                result_f = np.zeros([100, 100,3],dtype=np.uint8)
                result_f.fill(255) # or img[:] = 255
            
            aux.append(result_f)

        for idx, (_, im) in enumerate(zip(grid, aux )):
            # Iterating over the grid returns the Axes.
            ax = grid[cols*i+idx]
            ax.axis('off')
            ax.imshow(im,cmap='gray')

#     plt.show()
    data = get_img_from_fig(fig)
    plt.close()
    return data

The next function will concatenate the feature maps of the last layer, the actual feature map of n_batch image and the channel os the filter of the actual feature map

In [38]:
def concFig(fig1, fig2, layer, n_kn, n_batch, model):    #n_filters, img, layer, n_batch, n_ch=1):
    
    fig = plt.figure(figsize=(120., 54.))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(1,3),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                     )
    h = fig1.shape[0]
    w = fig1.shape[1]
    result_f = mpimg.imread('results/{}/feature_maps/fmp_layer_{}_num_kernel_{}_batch_{}.jpg'.format(model,layer,n_kn,n_batch))
    result_f =  cv2.resize(result_f, (w,h))
       
    aux = [fig1, result_f, fig2]
    for idx, (_, im) in enumerate(zip(grid, aux )):
        # Iterating over the grid returns the Axes.
        ax = grid[idx]
        ax.axis('off')
        ax.imshow(im, cmap='gray')

    data = get_img_from_fig(fig)
    plt.close()
    return data

This will take all the functions above and plot the visualization for all the layer that is required

In [39]:
def plot_layer_n_batch_n(layer, n_batch, cols, num_kernels, n_ch, model):    #n_filters, img, layer, n_batch, n_ch=1):
    
    fig = plt.figure(figsize=(120., 54. * num_kernels))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(num_kernels,1),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                     )
    fig1 = fmlayern_0(n_ch, cols, layer, n_batch, model)
    for i in range(num_kernels):
#         fig1 = fmlayern_0(n_ch, cols, layer, n_batch)
        fig2 = fmlayern_1(n_ch, cols, layer, i, model)
        row = concFig(fig1, fig2,layer,i,n_batch, model)
    
        ax = grid[i]
        ax.axis('off')
        ax.imshow(row, cmap='gray')
    
    directory = f'results/{model}/visualizations'
    if not os.path.exists(directory):
        os.makedirs(directory)

    plt.savefig("results/{}/visualizations/layer_{}_batch_{}.svg".format(model,layer,n_batch), bbox_inches='tight')
    plt.close()


The last one just take a bucle to reproduce this proccess in all the layers that are required

In [40]:
def plot_nlayer_vis(layers, n_batch, num_kernels, num_ch_perLayer, cols, model):
    # layers = [2,3]
    # n_batch = 3
    # num_kernels = [32,32]
    # num_ch_perlayer = [27,32]
    # cols = 6
    for i in range(n_batch):
        for idx, layer in enumerate(layers):
            print(f'Print ploting of {i} batch image in layer {layer}')
            plot_layer_n_batch_n(layer, i, cols, num_kernels[idx], num_ch_perlayer[idx], model)
        
    

## Excecutions

In [41]:
layers = [2,3,4,5,6]
n_batch = 1
num_kernels = [10,10,10,10,10]
num_ch_perlayer = [20,20,20,20,20]
cols = 8

In [42]:
get_fmlayer1('model12_init_30000_epochs_500', 3, 27)

In [43]:
model = 'model12_init_30000_epochs_500'
plot_nlayer_vis(layers, n_batch, num_kernels, num_ch_perlayer, cols, model)

Print ploting of 0 batch image in layer 2
Print ploting of 0 batch image in layer 3
Print ploting of 0 batch image in layer 4
Print ploting of 0 batch image in layer 5
Print ploting of 0 batch image in layer 6


In [73]:
get_fmlayer1('model6_dense_30000', 3, 27)

In [90]:
model = 'model6_dense_30000'
plot_nlayer_vis(layers, n_batch, num_k ernels, num_ch_perlayer, cols, model)

Print ploting of 0 batch image in layer 2
Print ploting of 0 batch image in layer 3
Print ploting of 0 batch image in layer 4
Print ploting of 0 batch image in layer 5
Print ploting of 0 batch image in layer 6


In [74]:
get_fmlayer1('model7_dense_30000_epochs_100', 3, 27)

In [91]:
model = 'model7_dense_30000_epochs_100'
plot_nlayer_vis(layers, n_batch, num_kernels, num_ch_perlayer, cols, model)

Print ploting of 0 batch image in layer 2
Print ploting of 0 batch image in layer 3
Print ploting of 0 batch image in layer 4
Print ploting of 0 batch image in layer 5
Print ploting of 0 batch image in layer 6


## Filters of Layer 1: Function to get all filter in layer 1 of a model

Get all the filters in 3d of a given layer 1

In [45]:
def get_filters_layer1(n_filters, cols, model):    #n_filters, img, layer, n_batch, n_ch=1):
    num_filter_to_show = n_filters

    fig = plt.figure(figsize=(40., 108.))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=((num_filter_to_show//cols)+1, cols ),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                     )
    
    for i in range((num_filter_to_show//cols)+1):
       
        aux = []
        for j in range(cols):
            
            if ((cols*i)+j < n_filters):
                #Load the image test and normalize
                result_f = mpimg.imread('results/{}/kernels/kn_layer_1_num_kernel_{}.jpg'.format(model, (cols*i)+j))
                cv2.cvtColor(result_f, cv2.COLOR_BGR2RGB)
                result_f = cv2.resize(result_f, (100,100))
            
            else:
            
                result_f = np.zeros([100,100,3],dtype=np.uint8)
                result_f.fill(255) # or img[:] = 255
            
            aux.append(result_f)

        for idx, (_, im) in enumerate(zip(grid, aux )):
            # Iterating over the grid returns the Axes.
            ax = grid[cols*i+idx]
            ax.axis('off')
            ax.imshow(im,cmap='gray')

#     plt.show()
    directory = f'results/{model}/visualizations_filters'
    if not os.path.exists(directory):
        os.makedirs(directory)

    plt.savefig("results/{}/visualizations_filters/layer_1.svg".format(model), bbox_inches='tight')
    plt.close()
    

This function is getting all the channels of a given kernel in a given layer for a specific batch feature map

In [19]:
def get_channels(n_filters, cols, layer, n_kn, model):    #n_filters, img, layer, n_batch, n_ch=1):
    num_filter_to_show = n_filters

    fig = plt.figure(figsize=(40., 108.))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=((num_filter_to_show//cols)+1, cols ),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                     )
    cont = 0
    for i in range((num_filter_to_show//cols)+1):
        
        aux = []
        for j in range(cols):
            
            if (cont < n_filters):
                #Load the image test and normalize
                result_f = mpimg.imread('results/{}/kernels/kn_layer_{}_num_kernel_{}_ch_{}.jpg'.format(model,layer,n_kn,cont))
                result_f = cv2.resize(result_f, (100,100))
                cont += 1
            
            else:
            
                result_f = np.zeros([100,100,3],dtype=np.uint8)
                result_f.fill(255) # or img[:] = 255
            
            aux.append(result_f)

        for idx, (_, im) in enumerate(zip(grid, aux )):
            # Iterating over the grid returns the Axes.
            ax = grid[cols*i+idx]
            ax.axis('off')
            ax.imshow(im,cmap='gray')

#     plt.show()
    data = get_img_from_fig(fig)
    plt.close()
    return data

In [24]:
def kernels_per_layer(num_kernels, n_channels, cols, layer, model):
    fig = plt.figure(figsize=(40., 27. *num_kernels))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(num_kernels,1),  # creates 2x2 grid of axes
                     axes_pad=0.1,  # pad between axes in inch.
                     )
    
    for idx in range(num_kernels):
        row = get_channels(n_channels, cols, layer, idx, model)
        ax = grid[idx]
        ax.axis('off')
        ax.imshow(row, cmap='gray')
    directory = f'results/{model}/visualizations_filters'
    if not os.path.exists(directory):
        os.makedirs(directory)

    plt.savefig("results/{}/visualizations_filters/layer_{}_num_kernels_{}_num_ch_{}.svg".format(model,layer, num_kernels, n_channels), bbox_inches='tight')
    plt.close()
    

## Excecutions

In [7]:
get_filters_layer1(27, 8, 'model9_full_tensor_30000_epochs_500')

In [47]:
kernels_per_layer(10, 40, 8, 3, 'model1_conv_1000')

In [25]:
kernels_per_layer(10, 27, 8, 3, 'model9_full_tensor_30000_epochs_500')

In [26]:
kernels_per_layer(5, 54, 8, 4, 'model9_full_tensor_30000_epochs_500')

In [27]:
kernels_per_layer(5, 54, 8, 5, 'model9_full_tensor_30000_epochs_500')

In [28]:
kernels_per_layer(5, 108, 8, 6, 'model9_full_tensor_30000_epochs_500')

In [29]:
get_filters_layer1(27, 8, 'model12_init_30000_epochs_500')
kernels_per_layer(10, 27, 8, 2, 'model12_init_30000_epochs_500')
kernels_per_layer(10, 27, 8, 3, 'model12_init_30000_epochs_500')
kernels_per_layer(5, 54, 8, 4, 'model12_init_30000_epochs_500')
kernels_per_layer(5, 54, 8, 5, 'model12_init_30000_epochs_500')
kernels_per_layer(5, 108, 8, 6, 'model12_init_30000_epochs_500')

In [46]:
get_filters_layer1(27, 8, 'model1_conv_1000')